# Indonesian Naive Bayes Sentiment Analyzer Demo
This notebook demonstrates the usage of the Indonesian Naive Bayes Sentiment Analyzer that combines multiple Indonesian lexicon resources.

In [ ]:
import pandas as pd
import numpy as np
from indonesian_naive_bayes_analyzer import IndonesianNaiveBayesAnalyzer, create_sample_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

## 1. Initialize the Analyzer
The analyzer will automatically load multiple Indonesian lexicon resources from GitHub repositories.

In [ ]:
# Initialize the Indonesian Naive Bayes Analyzer
print("Initializing Indonesian Naive Bayes Sentiment Analyzer...")
analyzer = IndonesianNaiveBayesAnalyzer()

# Display loaded lexicons
print(f"\nLoaded {len(analyzer.lexicons)} lexicon resources:")
for name, lexicon in analyzer.lexicons.items():
    print(f"- {name}: {len(lexicon)} entries")

## 2. Create Sample Dataset
We'll create a sample Indonesian sentiment dataset for demonstration.

In [ ]:
# Create sample dataset
texts, labels = create_sample_dataset()

# Create DataFrame for better visualization
df_sample = pd.DataFrame({
    'text': texts,
    'sentiment': labels
})

print(f"Sample dataset created with {len(df_sample)} examples")
print(f"\nSentiment distribution:")
print(df_sample['sentiment'].value_counts())

# Display sample data
print(f"\nSample data:")
display(df_sample.head(10))

## 3. Analyze Lexicon Coverage
Let's analyze how well our lexicons cover the vocabulary in the dataset.

In [ ]:
# Analyze lexicon coverage
coverage_results = analyzer.analyze_lexicon_coverage(texts)

# Visualize lexicon hits
lexicon_hits = coverage_results['lexicon_hits']
if lexicon_hits:
    plt.figure(figsize=(12, 6))
    plt.bar(lexicon_hits.keys(), lexicon_hits.values())
    plt.title('Lexicon Hits in Sample Dataset')
    plt.xlabel('Lexicon Source')
    plt.ylabel('Number of Hits')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
print(f"\nOverall lexicon coverage: {coverage_results['coverage']:.2%}")

## 4. Extract Lexicon Features
Let's examine the lexicon-based features extracted from sample texts.

In [ ]:
# Extract features for sample texts
sample_texts = [
    "Film ini sangat bagus dan menghibur sekali",
    "Saya tidak suka dengan film ini, sangat buruk",
    "Film yang biasa saja, tidak terlalu istimewa"
]

print("Lexicon Features Analysis:")
print("=" * 50)

feature_data = []
for i, text in enumerate(sample_texts):
    features = analyzer.extract_lexicon_features(text)
    feature_data.append(features)
    
    print(f"\nText {i+1}: {text}")
    print(f"Sentiment Score: {features['sentiment_score']:.3f}")
    print(f"Positive Words: {features['positive_count']}")
    print(f"Negative Words: {features['negative_count']}")
    print(f"Booster Words: {features['booster_count']}")
    print(f"Negation Words: {features['negation_count']}")

# Create DataFrame for features
df_features = pd.DataFrame(feature_data)
df_features['text'] = sample_texts

# Visualize features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sentiment scores
axes[0,0].bar(range(len(sample_texts)), df_features['sentiment_score'])
axes[0,0].set_title('Sentiment Scores')
axes[0,0].set_xlabel('Text Index')
axes[0,0].set_ylabel('Score')

# Word counts
word_counts = df_features[['positive_count', 'negative_count', 'booster_count', 'negation_count']]
word_counts.plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Word Counts by Type')
axes[0,1].set_xlabel('Text Index')
axes[0,1].legend()

# Positive vs Negative scores
axes[1,0].scatter(df_features['positive_score'], df_features['negative_score'])
axes[1,0].set_title('Positive vs Negative Scores')
axes[1,0].set_xlabel('Positive Score')
axes[1,0].set_ylabel('Negative Score')

# Feature correlation
feature_cols = ['positive_score', 'negative_score', 'sentiment_score', 'positive_count', 'negative_count']
corr_matrix = df_features[feature_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1,1])
axes[1,1].set_title('Feature Correlation')

plt.tight_layout()
plt.show()

## 5. Train the Naive Bayes Model
Now let's train the Naive Bayes model using the combined lexicon and TF-IDF features.

In [ ]:
# Train the model
print("Training the Indonesian Naive Bayes model...")
training_results = analyzer.train(texts, labels, test_size=0.3, random_state=42)

print(f"\nTraining Results:")
print(f"Accuracy: {training_results['accuracy']:.4f}")

## 6. Test Predictions
Let's test the trained model on various Indonesian texts.

In [ ]:
# Test texts
test_texts = [
    "Film ini sangat bagus dan menghibur sekali, saya suka banget!",
    "Saya tidak suka dengan film ini, sangat buruk dan mengecewakan",
    "Film yang biasa saja, tidak terlalu istimewa tapi lumayan",
    "Ceritanya hebat dan aktingnya luar biasa, sangat memuaskan",
    "Sangat membosankan, saya tidak akan merekomendasikan film ini",
    "Film yang cukup bagus untuk ditonton bersama keluarga",
    "Tidak ada yang menarik dari film ini, membuang waktu saja",
    "Saya sangat terkesan dengan kualitas produksinya yang mantap"
]

print("Sentiment Analysis Results:")
print("=" * 80)

results = []
for i, text in enumerate(test_texts):
    result = analyzer.predict_sentiment(text)
    results.append(result)
    
    print(f"\nText {i+1}: {text}")
    print(f"Predicted Sentiment: {result['sentiment']}")
    print(f"Confidence: {result['confidence']:.3f}")
    print(f"Lexicon Score: {result['lexicon_score']:.3f}")
    print(f"Positive Words: {result['positive_words']}, Negative Words: {result['negative_words']}")
    print("-" * 80)

# Create results DataFrame
df_results = pd.DataFrame({
    'text': test_texts,
    'predicted_sentiment': [r['sentiment'] for r in results],
    'confidence': [r['confidence'] for r in results],
    'lexicon_score': [r['lexicon_score'] for r in results]
})

# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Sentiment distribution
sentiment_counts = df_results['predicted_sentiment'].value_counts()
axes[0].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%')
axes[0].set_title('Predicted Sentiment Distribution')

# Confidence scores
axes[1].bar(range(len(test_texts)), df_results['confidence'])
axes[1].set_title('Prediction Confidence')
axes[1].set_xlabel('Text Index')
axes[1].set_ylabel('Confidence')
axes[1].set_ylim(0, 1)

# Lexicon scores vs confidence
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'blue'}
for sentiment in df_results['predicted_sentiment'].unique():
    mask = df_results['predicted_sentiment'] == sentiment
    axes[2].scatter(df_results[mask]['lexicon_score'], df_results[mask]['confidence'], 
                   c=colors.get(sentiment, 'gray'), label=sentiment, alpha=0.7)

axes[2].set_title('Lexicon Score vs Confidence')
axes[2].set_xlabel('Lexicon Score')
axes[2].set_ylabel('Confidence')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Model Performance Analysis
Let's analyze the model's performance in more detail.

In [ ]:
# Display detailed classification report
if 'classification_report' in training_results:
    report = training_results['classification_report']
    
    print("Detailed Classification Report:")
    print("=" * 50)
    
    # Create DataFrame for better visualization
    metrics_df = pd.DataFrame(report).transpose()
    
    # Remove support column for cleaner display
    if 'support' in metrics_df.columns:
        display_df = metrics_df.drop('support', axis=1)
    else:
        display_df = metrics_df
    
    display(display_df.round(3))
    
    # Visualize metrics
    if len(display_df) > 3:  # Ensure we have class-specific metrics
        class_metrics = display_df.iloc[:-3]  # Exclude macro avg, weighted avg, accuracy
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Precision
        class_metrics['precision'].plot(kind='bar', ax=axes[0], color='skyblue')
        axes[0].set_title('Precision by Class')
        axes[0].set_ylabel('Precision')
        axes[0].set_ylim(0, 1)
        
        # Recall
        class_metrics['recall'].plot(kind='bar', ax=axes[1], color='lightcoral')
        axes[1].set_title('Recall by Class')
        axes[1].set_ylabel('Recall')
        axes[1].set_ylim(0, 1)
        
        # F1-score
        class_metrics['f1-score'].plot(kind='bar', ax=axes[2], color='lightgreen')
        axes[2].set_title('F1-Score by Class')
        axes[2].set_ylabel('F1-Score')
        axes[2].set_ylim(0, 1)
        
        plt.tight_layout()
        plt.show()

print(f"\nModel Summary:")
print(f"- Lexicon sources: {len(analyzer.lexicons)}")
print(f"- Total lexicon entries: {sum(len(lex) for lex in analyzer.lexicons.values())}")
print(f"- Training accuracy: {training_results['accuracy']:.4f}")
print(f"- Feature combination: Lexicon + TF-IDF")

## 8. Save and Load Model
Demonstrate how to save and load the trained model.

In [ ]:
# Save the model
model_path = 'indonesian_nb_sentiment_model.pkl'
analyzer.save_model(model_path)

# Load the model (demonstration)
new_analyzer = IndonesianNaiveBayesAnalyzer()
new_analyzer.load_model(model_path)

# Test the loaded model
test_text = "Film ini sangat bagus dan menghibur"
result = new_analyzer.predict_sentiment(test_text)

print(f"Testing loaded model:")
print(f"Text: {test_text}")
print(f"Sentiment: {result['sentiment']}")
print(f"Confidence: {result['confidence']:.3f}")

print(f"\nModel successfully saved and loaded!")